### 1 - Klasmen Club

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL untuk halaman tabel Premier League di Sky Sports
url = "https://www.skysports.com/premier-league-table"

# Menambahkan headers untuk permintaan
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Mengambil konten dari halaman
response = requests.get(url, headers=headers)

# Memeriksa apakah permintaan berhasil
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    # Mencari tabel dengan kelas "sdc-site-table" dan ID "sdc-site-table--22"
    table = soup.find("table", class_="sdc-site-table", id="sdc-site-table--22")

    # List untuk menyimpan data
    premier_league_table = []

    if table:
        for row in table.find_all("tr")[1:]:  # Menghindari header
            cols = row.find_all("td")
            if len(cols) > 1:
                try:
                    Club = cols[1].text.strip()  # Nama klub
                    Played = cols[2].text.strip()  # Jumlah pertandingan
                    Won = cols[3].text.strip()  # Menang
                    Drawn = cols[4].text.strip()  # Seri
                    Lost = cols[5].text.strip()  # Kalah
                    GF = cols[6].text.strip()  # Goals For
                    GA = cols[7].text.strip()  # Goals Against
                    GD = cols[8].text.strip()  # Goal Difference
                    Points = cols[9].text.strip()  # Poin

                    premier_league_table.append({
                        'Club': Club,
                        'Played': Played,
                        'Won': Won,
                        'Drawn': Drawn,
                        'Lost': Lost,
                        'GF': GF,
                        'GA': GA,
                        'GD': GD,
                        'Points': Points
                    })

                except (IndexError, AttributeError):
                    continue  # Lewati jika terjadi error

    # Mengubah list menjadi DataFrame
    df_premier_league_table = pd.DataFrame(premier_league_table)

    # Menyimpan DataFrame ke file CSV tanpa menyertakan indeks
    df_premier_league_table.to_csv('premier_league_table.csv', index=False)

    # Menampilkan DataFrame tanpa indeks
    df_premier_league_table.index = range(1, len(df_premier_league_table) + 1)
    print(df_premier_league_table.to_string(index=True))






                        Club Played Won Drawn Lost  GF  GA   GD Points
1                  Liverpool     15  11     3    1  31  13  +18     36
2                    Chelsea     16  10     4    2  37  19  +18     34
3                    Arsenal     16   8     6    2  29  15  +14     30
4          Nottingham Forest     16   8     4    4  21  19   +2     28
5                Aston Villa     17   8     4    5  26  26    0     28
6            Manchester City     17   8     3    6  29  25   +4     27
7                Bournemouth     16   7     4    5  24  21   +3     25
8                     Fulham     16   6     6    4  24  22   +2     24
9   Brighton and Hove Albion     16   6     6    4  26  25   +1     24
10         Tottenham Hotspur     16   7     2    7  36  19  +17     23
11                 Brentford     16   7     2    7  32  30   +2     23
12          Newcastle United     16   6     5    5  23  21   +2     23
13         Manchester United     16   6     4    6  21  19   +2     22
14    

### 2 - Market Value Tim

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL halaman Transfermarkt
url = "https://www.transfermarkt.com/premier-league/marktwerteverein/wettbewerb/GB1"

# Mengirimkan permintaan ke URL
response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
soup = BeautifulSoup(response.content, 'html.parser')

# Mencari tabel dengan class "items"
table = soup.find('table', {'class': 'items'})

# Menyimpan hasil ke list
club_value_data = []

# Iterasi melalui baris tabel
for row in table.find('tbody').find_all('tr'):
    # Mendapatkan peringkat
    rank = row.find('td', {'class': 'zentriert'}).text.strip()

    # Mendapatkan nama klub
    Club = row.find('td', {'class': 'hauptlink'}).find('a').text.strip()

    # Mendapatkan total value
    value = row.find_all('td', {'class': 'rechts'})[1].text.strip()

    # Hapus angka nol dari total nilai (kalau ada)
    value_clean = value.replace('.00', '').replace('€', '').strip()

    # Menyimpan data ke list
    club_value_data.append({
        'Rank': rank,
        'Club': Club,
        'Total Value': value_clean
    })

# Mengubah list menjadi DataFrame
df_mvclubs = pd.DataFrame(club_value_data)

# Menampilkan DataFrame tanpa indeks
print(df_mvclubs.to_string(index=False))

# Menyimpan DataFrame ke file CSV tanpa menyertakan indeks
df_mvclubs.to_csv('club_value.csv', index=False)

Rank                    Club Total Value
   1         Manchester City      1.19bn
   2              Arsenal FC      1.15bn
   3              Chelsea FC      1.01bn
   4            Liverpool FC        967m
   5       Manchester United     794.65m
   6       Tottenham Hotspur     780.30m
   7        Newcastle United     639.35m
   8  Brighton & Hove Albion     606.60m
   9             Aston Villa     601.95m
  10         West Ham United     439.30m
  11       Nottingham Forest        437m
  12          Crystal Palace     424.90m
  13            Brentford FC     417.93m
  14         AFC Bournemouth     391.35m
  15 Wolverhampton Wanderers     377.80m
  16               Fulham FC        347m
  17              Everton FC     343.10m
  18          Southampton FC     306.70m
  19          Leicester City     275.80m
  20            Ipswich Town     235.80m


### 3 - Goal Assist Player

In [ ]:
import requests
from bs4 import BeautifulSoup

# Fungsi untuk mengambil data dari satu halaman
def get_player_data(url):
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(response.content, 'html.parser')
    table_body = soup.find('tbody', {'class': 'stats-table__container statsTableContainer'})

    players_data = []

    for row in table_body.find_all('tr', class_='table__row'):
        rank = row.find('td', {'class': 'stats-table__rank'}).text.strip().replace('.', '')
        player_name = row.find('a', {'class': 'playerName'}).text.strip()
        club_name = row.find('a', {'class': 'stats-table__cell-icon-align'}).text.strip()
        goals = row.find('td', {'class': 'stats-table__main-stat'}).text.strip()

        players_data.append({
            'Rank': rank,
            'Player Name': player_name,
            'Club': club_name,
            'Goals': goals
        })

    return players_data

# URL halaman Premier League untuk daftar top scorer
url = "https://www.premierleague.com/stats/top/players/goals?se=719"

all_players_data = []

# Mengambil data dari halaman sampai tidak ada lagi halaman berikutnya
while True:
    print(f"Fetching data from: {url}")
    players_data = get_player_data(url)
    all_players_data.extend(players_data)

    # Mencari tombol "Next"
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(response.content, 'html.parser')

    # Mencetak HTML di bagian bawah untuk melihat navigasi
    pagination_section = soup.find('div', class_='pagination')
    if pagination_section:
        print("Pagination Section:")
        print(pagination_section.prettify())

    # Mencari tombol "Next"
    next_button = pagination_section.find('a', text='Next') or pagination_section.find('a', string='>')

    # Jika kita menemukan tombol "Next"
    if next_button:
        print("Next button found.")
        url = "https://www.premierleague.com" + next_button['href']
        print(f"Next page URL: {url}")
    else:
        print("No next button found.")
        break  # Jika tidak ada tombol "Next", keluar dari loop

# Menampilkan hasil
for player in all_players_data:
    print(player)


### 4 - Rata Rata Supporter Yang Hadir


In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL halaman yang akan di-scrape
url = 'https://www.transfermarkt.com/premier-league/besucherzahlen/wettbewerb/GB1'

# Meminta halaman web
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')

# Mencari tabel yang mengandung data yang diinginkan
table = soup.find('table', class_='items')

# List untuk menyimpan hasil
attendance = []

# Iterasi setiap baris pada tabel
for row in table.find('tbody').find_all('tr'):
    # Mencari semua kolom dalam baris
    cols = row.find_all('td')
    if len(cols) > 4:
        # Mendapatkan data: nama klub, stadion, kapasitas, rata-rata kehadiran
        club = cols[4].text.strip()
        stadium = cols[3].text.strip()
        capacity = cols[5].text.strip()
        average_attendance = cols[7].text.strip()

        # Menyimpan ke dalam list
        attendance.append({
            'Club': club,
            'Stadium': stadium,
            'Capacity': capacity,
            'Average Attendance': average_attendance
        })

# Mengubah list menjadi DataFrame
df_attendance = pd.DataFrame(attendance)

# Menampilkan DataFrame tanpa indeks
print(df_attendance.to_string(index=False))

# Menyimpan DataFrame ke file CSV tanpa menyertakan indeks
df_attendance.to_csv('Attendance.csv', index=False)


                   Club                   Stadium Capacity Average Attendance
      Manchester United              Old Trafford   74.879             73.861
        West Ham United            London Stadium   62.500             62.271
      Tottenham Hotspur Tottenham Hotspur Stadium   62.850             61.339
           Liverpool FC                   Anfield   60.725             60.483
             Arsenal FC          Emirates Stadium   60.704             60.294
       Newcastle United            St James' Park   52.338             52.211
            Aston Villa                Villa Park   42.682             42.682
        Manchester City            Etihad Stadium   53.400             54.748
             Chelsea FC           Stamford Bridge   40.853             40.705
             Everton FC             Goodison Park   39.571             39.010
         Leicester City        King Power Stadium   32.273             31.697
         Southampton FC         St Mary's Stadium   32.384      

### 5 - Injured Player 


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL halaman yang akan di-scrape
url = 'https://www.transfermarkt.com/premier-league/verletztespieler/wettbewerb/GB1'

# Meminta halaman web dengan user-agent
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')

# Mencari tabel yang mengandung data yang diinginkan
table = soup.find('table', class_='items')

# List untuk menyimpan hasil
injured_player = []

# Iterasi setiap baris pada tabel
for row in table.find('tbody').find_all('tr'):
    # Mencari semua kolom dalam baris
    cols = row.find_all('td')
    if len(cols) > 6:  # Memastikan cukup kolom yang tersedia
        # Mendapatkan data: nama pemain, klub, jenis cedera, sampai kapan (until), market value
        player = cols[2].text.strip()  # Nama pemain
        
        # Mengambil nama klub dari atribut title di tag <a>
        club_link = cols[4].find('a')  # Cek apakah tag <a> ada
        club = club_link['title'].strip() if club_link and 'title' in club_link.attrs else 'N/A'  # Mengambil dari atribut 'title' atau 'N/A' jika tidak ada
        
        injury = cols[5].text.strip()  # Jenis cedera
        until = cols[6].text.strip() if cols[5].text.strip() else 'Unknown'
        market_value = cols[7].text.strip()  # Market value pemain

        # Menyimpan ke dalam list
        injured_player.append({
            'Player': player,
            'Club': club,
            'Injury': injury,
            'Until': until,
            'Market Value': market_value
        })

# Mengubah list menjadi DataFrame
df_injured_player = pd.DataFrame(injured_player)

# Menampilkan DataFrame tanpa indeks
print(df_injured_player.to_string(index=False))

# Menyimpan DataFrame ke file CSV tanpa menyertakan indeks
df_injured_player.to_csv('injured.csv', index=False)



### 6 Statistik Chance Convertion Rate Tim


In [9]:
import requests
from bs4 import BeautifulSoup
import re  # Mengimpor regex
import pandas as pd

# URL halaman yang akan di-scrape
url = 'https://www.transfermarkt.com/premier-league/chancenverwertung/wettbewerb/GB1/saison_id/2024/plus/1/sort/schuesse_auf_tor.desc'

# Meminta halaman web dengan user-agent
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')

# Mencari tabel yang mengandung data yang diinginkan
table = soup.find('table', class_='items')

# List untuk menyimpan hasil
chance_coveration= []

# Iterasi setiap baris pada tabel
for row in table.find('tbody').find_all('tr'):
    # Mencari semua kolom dalam baris
    cols = row.find_all('td')
    if len(cols) > 5:  # Memastikan cukup kolom yang tersedia
        # Mendapatkan data: nama klub, tembakan tidak tepat sasaran, tembakan tepat sasaran, total gol, rasio gol
        club_with_position = cols[1].text.strip()  # Ambil teks lengkap
        
        # Menghapus angka dan kata 'th', 'st', 'nd', 'rd' dari nama klub
        club = re.sub(r'\s*\d+(st|nd|rd|th)?\s*position?', '', club_with_position).strip()

        # Tembakan tidak tepat sasaran
        shoot_off_target = cols[2].text.strip()
        
        # Tembakan tepat sasaran
        shoot_on_target = cols[3].text.strip()
        
        # Total gol
        total_goal = cols[4].text.strip()
        
        # Rasio gol
        goal_ratio = cols[5].text.strip()

        # Menyimpan ke dalam list
        chance_coveration.append({
            'Club': club,
            'Shoot Off Target': shoot_off_target,
            'Shoot On Target': shoot_on_target,
            'Total Goal': total_goal,
            'Goal Ratio': goal_ratio
        })

# Mengubah list menjadi DataFrame
df_chance_coveration = pd.DataFrame(chance_coveration)

# Menampilkan DataFrame tanpa indeks
print(df_chance_coveration.to_string(index=False))

# Menyimpan DataFrame ke file CSV tanpa menyertakan indeks
df_chance_coveration.to_csv('chance_coveration.csv', index=False)

                   Club Shoot Off Target Shoot On Target Total Goal Goal Ratio
        Manchester City              384             282         28      7.3 %
        AFC Bournemouth              369             270         24      6.5 %
             Chelsea FC              343             256         37     10.8 %
        West Ham United              330             241         21      6.4 %
      Tottenham Hotspur              316             236         36     11.4 %
 Brighton & Hove Albion              313             233         26      8.3 %
           Liverpool FC              308             232         31     10.1 %
             Arsenal FC              307             226         29      9.4 %
         Crystal Palace              306             225         17      5.6 %
       Newcastle United              305             218         23      7.5 %
            Aston Villa              290             209         24      8.3 %
      Manchester United              282            

### 7 - Tim Possesion

In [10]:
import requests
from bs4 import BeautifulSoup
import re  # Mengimpor regex
import pandas as pd

# URL halaman yang akan di-scrape
url = 'https://www.fctables.com/england/premier-league/ball-possession/'

# Meminta halaman web dengan user-agent
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')

# Mencari tabel yang mengandung data yang diinginkan
table = soup.find('table', class_='table stage-table table-striped table-bordered table-hover table-condensed responsive')

# List untuk menyimpan hasil
tim_possesion= []

# Iterasi setiap baris pada tabel
for row in table.find('tbody').find_all('tr'):
    # Mencari semua kolom dalam baris
    cols = row.find_all('td')
    if len(cols) > 4:  # Memastikan cukup kolom yang tersedia
        # Mengambil nama klub
        club = cols[1].text.strip()

        # Mengambil data yang diinginkan
        games_played = cols[2].text.strip()
        avg_possession = cols[3].text.strip()
        opponent_possession = cols[4].text.strip()

        # Menyimpan ke dalam list
        tim_possesion.append({
            'Club': club,
            'Games Played': games_played,
            'Average Possession Club': avg_possession,
            'Average Opponent Possession': opponent_possession
        })

# Mengubah list menjadi DataFrame
df_tim_possesion = pd.DataFrame(tim_possesion)

# Menampilkan DataFrame tanpa indeks
print(df_tim_possesion.to_string(index=False))

# Menyimpan DataFrame ke file CSV tanpa menyertakan indeks
df_tim_possesion.to_csv('tim_possession.csv', index=False)


                   Club Games Played Average Possession Club Average Opponent Possession
        Manchester City           16                  63.06%                      36.94%
              Tottenham           16                  58.44%                      41.56%
              Liverpool           15                  57.40%                      42.60%
                Chelsea           16                  56.94%                      43.06%
                Arsenal           16                  54.50%                      45.50%
               Brighton           16                  53.88%                      46.13%
      Manchester United           16                  53.88%                      46.13%
            Southampton           16                  53.19%                      46.81%
                 Fulham           16                  50.88%                      49.13%
            Aston Villa           16                  50.56%                      49.44%
       Newcastle Unit

### 8 Fixtures 


In [ ]:
import pandas as pd

# Membaca file JSON
data = pd.read_json('en.1.json')

# Menyimpan data sebagai file CSV
data.to_csv('fixtures.csv', index=False)




In [ ]:
import pandas as pd

# Memuat dataset
attendance_df = pd.read_csv('Attendance.csv')
chance_conversion_df = pd.read_csv('chance_coveration.csv')
club_value_df = pd.read_csv('club_value.csv')
injured_df = pd.read_csv('injured.csv')
league_table_df = pd.read_csv('premier_league_table.csv')
possession_df = pd.read_csv('tim_possession.csv')

# Memeriksa kolom di setiap dataset
print("Kolom di Attendance:", attendance_df.columns)
print("Kolom di Chance Conversion:", chance_conversion_df.columns)
print("Kolom di Club Value:", club_value_df.columns)
print("Kolom di Injured:", injured_df.columns)
print("Kolom di League Table:", league_table_df.columns)
print("Kolom di Possession:", possession_df.columns)


In [ ]:
import pandas as pd

# Memuat dataset tanpa injured.csv
attendance_df = pd.read_csv('Attendance.csv')
chance_conversion_df = pd.read_csv('chance_coveration.csv')
club_value_df = pd.read_csv('club_value.csv')
league_table_df = pd.read_csv('premier_league_table.csv')
possession_df = pd.read_csv('tim_possession.csv')

# Kamus untuk normalisasi nama klub
club_name_mapping = {
    "Man United": "Manchester United",
    "Man City": "Manchester City",
    "Spurs": "Tottenham",
    "Newcastle Utd": "Newcastle",
    "Wolves": "Wolves",
    "Arsenal" : "Arsenal",
    "Bournemouth" : "Bournemouth",
    "Brentford" : "Brentford", 
    "Brighton" : "Brighton",
    "Brighton and Hove Albion" : "Brighton",
    "Chelsea" : "Chelsea",
    "Everton" : "Everton", 
    "Leicester" : "Leicester", 
    "Liverpool" : "Liverpool",
    "Southampton" : "Southampton",


    # Tambahkan variasi lain yang ditemukan
}

# Fungsi untuk menerapkan normalisasi nama klub
def normalize_club_names(df, club_col='Club'):
    df[club_col] = df[club_col].replace(club_name_mapping)
    return df

# Terapkan normalisasi ke setiap dataset
attendance_df = normalize_club_names(attendance_df)
chance_conversion_df = normalize_club_names(chance_conversion_df)
club_value_df = normalize_club_names(club_value_df)
league_table_df = normalize_club_names(league_table_df)
possession_df = normalize_club_names(possession_df)

# Lakukan penggabungan dataset setelah normalisasi nama klub
merged_df = pd.merge(attendance_df, chance_conversion_df, on='Club', how='outer')
merged_df = pd.merge(merged_df, club_value_df, on='Club', how='outer')
merged_df = pd.merge(merged_df, league_table_df, on='Club', how='outer')
merged_df = pd.merge(merged_df, possession_df, on='Club', how='outer')

# Pra-pemrosesan data
merged_df.fillna(0, inplace=True)  # Mengisi nilai kosong dengan 0

# Konversi tipe data kolom numerik sesuai kebutuhan
numerical_cols = merged_df.select_dtypes(include=['float64', 'int64']).columns
for col in numerical_cols:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')

# Menghapus data duplikat
merged_df.drop_duplicates(inplace=True)

# Mengurutkan data berdasarkan kolom 'Points' dan 'GD'
# Urutan menurun pada kedua kolom untuk memastikan posisi dengan poin dan GD yang lebih besar berada di atas
sorted_df = merged_df.sort_values(by=['Points', 'GD'], ascending=[False, False])

# Menampilkan hasil pengurutan
print("Data setelah diurutkan berdasarkan Points dan GD:\n", sorted_df[['Club', 'Points', 'GD']].head(10))

# Menyimpan hasil ke file baru
sorted_df.to_csv('merged_premier_league_data.csv', index=False)



Data setelah diurutkan berdasarkan Points dan GD:
                  Club  Points    GD
20          Liverpool    36.0  18.0
9             Chelsea    34.0  18.0
1             Arsenal    30.0  14.0
25  Nottingham Forest    28.0   2.0
22    Manchester City    27.0   5.0
4         Bournemouth    25.0   3.0
3         Aston Villa    25.0  -1.0
14             Fulham    24.0   2.0
7            Brighton    24.0   1.0
29  Tottenham Hotspur    23.0  17.0


In [ ]:
# Import necessary libraries
import pandas as pd

# Load datasets
stats_df = pd.read_csv('merged_premier_league_data.csv')
fixtures_df = pd.read_csv('2024_premier_league_fixtures_df.csv')

# Kamus untuk normalisasi nama klub
club_name_mapping = {
    "Man United": "Manchester United",
    "Man City": "Manchester City",
    "Tottenham Hotspur": "Tottenham",
    "Newcastle United": "Newcastle",
    "Wolverhampton Wanderers": "Wolves",
    "Arsenal FC": "Arsenal",
    "AFC Bournemouth": "Bournemouth",
    "Brentford FC": "Brentford",
    "Brighton & Hove Albion": "Brighton",
    "Chelsea FC": "Chelsea",
    "Everton FC": "Everton",
    "Fulham FC": "Fulham",
    "Ipswich Town": "Ipswich",
    "Leicester City": "Leicester",
    "Liverpool FC": "Liverpool",
    "Southampton FC": "Southampton",
    "West Ham United": "West Ham"
}

# Kamus untuk normalisasi nama stadion
venue_name_mapping = {
    "AMEX Stadium": "American Express Stadium",
    "St James' Park": "St. James' Park",
    "St Mary's Stadium": "St. Mary's Stadium"
}

# Normalisasi nama klub di data statistik
stats_df['Club'] = stats_df['Club'].replace(club_name_mapping)

# Normalisasi nama stadion di data statistik
stats_df['Stadium'] = stats_df['Stadium'].replace(venue_name_mapping)

# Buat daftar venue unik dengan ID dari data fixtures
venue_ids = fixtures_df[['Venue', 'Home Team ID']].dropna().drop_duplicates()
venue_ids = venue_ids.rename(columns={'Venue': 'Stadium', 'Home Team ID': 'Venue ID'})

# Buat daftar ID tim dengan nama tim
home_teams = fixtures_df[['Home Team ID', 'Home Team']].dropna().drop_duplicates()
away_teams = fixtures_df[['Away Team ID', 'Away Team']].dropna().drop_duplicates()

# Rename columns for consistency before combining
home_teams.columns = ['Team ID', 'Team']
away_teams.columns = ['Team ID', 'Team']

# Gabungkan daftar tim kandang dan tandang, dan hilangkan duplikasi
team_list = pd.concat([home_teams, away_teams]).drop_duplicates().reset_index(drop=True)

# Gabungkan daftar tim dengan data statistik untuk menambahkan ID tim yang sesuai
merged_stats = stats_df.merge(team_list, left_on='Club', right_on='Team', how='left')

# Gabungkan data statistik yang sudah digabungkan dengan daftar venue untuk menambahkan ID venue yang sesuai
final_stats = merged_stats.merge(venue_ids, left_on='Stadium', right_on='Stadium', how='left')

# Memastikan kolom 'Team ID' dan 'Venue ID' terisi lengkap dengan melakukan pengecekan
missing_team_ids = final_stats[final_stats['Team ID'].isna()]['Club'].unique()
missing_venue_ids = final_stats[final_stats['Venue ID'].isna()]['Stadium'].unique()

# Menampilkan nama tim dan stadion yang tidak memiliki ID setelah normalisasi
print("Nama tim yang tidak memiliki Team ID setelah normalisasi:", missing_team_ids)
print("Stadion yang tidak memiliki Venue ID setelah normalisasi:", missing_venue_ids)

# Simpan data akhir ke file CSV
final_stats.to_csv('final_stats_with_team_and_venue_id.csv', index=False)
print("Final merged data with Team and Venue IDs saved as 'final_stats_with_team_and_venue_id.csv'")

# Tampilkan hasil akhir dari data yang digabungkan
print("\nFinal Merged Data with Team and Venue IDs:")
print(final_stats.head())


In [ ]:
# Import necessary libraries
import pandas as pd

# Load datasets
stats_df = pd.read_csv('merged_premier_league_data.csv')
fixtures_df = pd.read_csv('2024_premier_league_fixtures_df.csv')

# Create unique venue list with ID from fixtures data
venue_ids = fixtures_df[['Venue', 'Home Team ID']].dropna().drop_duplicates()
venue_ids = venue_ids.rename(columns={'Venue': 'Stadium', 'Home Team ID': 'Venue ID'})

# Create team list to ensure IDs match correctly
home_teams = fixtures_df[['Home Team ID', 'Home Team']].dropna().drop_duplicates()
away_teams = fixtures_df[['Away Team ID', 'Away Team']].dropna().drop_duplicates()

# Rename columns for consistency before combining
home_teams.columns = ['Team ID', 'Team']
away_teams.columns = ['Team ID', 'Team']

# Combine home and away teams and remove duplicates
team_list = pd.concat([home_teams, away_teams]).drop_duplicates().reset_index(drop=True)

# Merge team list into stats data to add consistent Team IDs
merged_stats = stats_df.merge(team_list, left_on='Club', right_on='Team', how='left')

# Now merge venue information to add Venue ID based on Stadium names
final_stats = merged_stats.merge(venue_ids, left_on='Stadium', right_on='Stadium', how='left')

# Save the final merged data to a CSV file
final_stats.to_csv('final_stats_with_team_and_venue_id.csv', index=False)
print("Final merged data with Team and Venue IDs saved as 'final_stats_with_team_and_venue_id.csv'")

# Display the final merged data
print("\nFinal Merged Data with Team and Venue IDs:")
print(final_stats.head())


In [ ]:
# Import necessary libraries
import pandas as pd

# Load the primary dataset with team and venue IDs
data = pd.read_csv('final_stats_with_team_and_venue_id.csv')
fixtures = pd.read_csv('2024_premier_league_fixtures_df.csv')

# Check column names to confirm 'Team ID' and 'Fixture ID' exist
print("Columns in data:", data.columns)
print("Columns in fixtures:", fixtures.columns)

# Function to clean 'bn' and 'm' suffixes
def convert_currency(value):
    if isinstance(value, str):
        value = value.replace(' ', '').replace('bn', 'e9').replace('m', 'e6')
        try:
            return float(eval(value))
        except:
            return None
    return value

# Function to remove percentage symbol and convert to float
def convert_percentage(value):
    if isinstance(value, str) and '%' in value:
        return float(value.replace('%', '')) / 100
    return value

# Apply cleaning functions to relevant columns
data['Total Value'] = data['Total Value'].apply(convert_currency)
data['Goal Ratio'] = data['Goal Ratio'].apply(convert_percentage)
data['Average Possession Club'] = data['Average Possession Club'].apply(convert_percentage)
data['Average Opponent Possession'] = data['Average Opponent Possession'].apply(convert_percentage)

# Convert columns to numeric if they are not already
data['Total Value'] = pd.to_numeric(data['Total Value'], errors='coerce')
data['Goal Ratio'] = pd.to_numeric(data['Goal Ratio'], errors='coerce')
data['Average Possession Club'] = pd.to_numeric(data['Average Possession Club'], errors='coerce')
data['Average Opponent Possession'] = pd.to_numeric(data['Average Opponent Possession'], errors='coerce')

# Initialize a list to capture matches for each team along with fixture details
team_matches = []

# Iterate over each row in fixtures to record matches based on team IDs
for _, row in fixtures.iterrows():
    # Record home team match
    team_matches.append({
        'Team ID': row['Home Team ID'], 
        'Fixture ID': row['Fixture ID'], 
        'Match Type': 'Home',
        'Game Date': row['Game Date'],   
        'Stadium': row['Venue']
    })
    # Record away team match
    team_matches.append({
        'Team ID': row['Away Team ID'], 
        'Fixture ID': row['Fixture ID'], 
        'Match Type': 'Away',
        'Game Date': row['Game Date'],   
        'Stadium': row['Venue']
    })

# Create a DataFrame for the team match data with Fixture ID, Match Type, Game Date, and Stadium
matches_df = pd.DataFrame(team_matches)

# Verify column names in matches_df before merging
print("Columns in matches_df:", matches_df.columns)

# Ensure 'Team ID' and 'Fixture ID' have consistent data types in both DataFrames
data['Team ID'] = pd.to_numeric(data['Team ID'], errors='coerce')
matches_df['Team ID'] = pd.to_numeric(matches_df['Team ID'], errors='coerce')
matches_df['Fixture ID'] = pd.to_numeric(matches_df['Fixture ID'], errors='coerce')

# Merge this information with the main dataset based on Team ID
data = data.merge(matches_df, on='Team ID', how='left')

# Save the cleaned data with new columns to a new CSV file
cleaned_file_path = 'cleaned_final_stats_with_team_and_venue_id.csv'
data.to_csv(cleaned_file_path, index=False)

print("Data cleaned and saved as 'cleaned_final_stats_with_team_and_venue_id.csv'")
print("\nSample of cleaned data with added columns:")
print(data.head())


In [ ]:
# Import pustaka yang diperlukan
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Memuat dataset utama dengan Team ID dan Venue ID
data = pd.read_csv('cleaned_final_stats_with_team_and_venue_id.csv')
# Inisialisasi MinMaxScaler
scaler = MinMaxScaler()

# Definisikan kolom-kolom yang akan dinormalisasi dengan Min-Max Scaler
columns_to_scale = [
    'Average Attendance', 'Shoot Off Target', 'Shoot On Target', 'Total Goal', 'Goal Ratio', 'Rank', 
    'Total Value', 'Played', 'Won', 'Drawn', 'Lost', 'GF', 'GA', 'GD', 'Points', 
    'Games Played', 'Average Possession Club', 'Average Opponent Possession', 'Total Value' 
]

# Terapkan MinMaxScaler hanya pada kolom yang ditentukan
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

# Menyimpan data yang telah dibersihkan dan dinormalisasi ke file CSV baru
scaled_file_path = 'scaled_final_stats_with_team_and_venue_id.csv'
data.to_csv(scaled_file_path, index=False)

print("Data telah dinormalisasi dan disimpan sebagai 'scaled_final_stats_with_team_and_venue_id.csv'")
print("\nContoh data yang telah dinormalisasi dengan Min-Max Scaler:")
print(data.head())

In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the cleaned dataset
data = pd.read_csv('scaled_final_stats_with_team_and_venue_id.csv')

# 1. Melihat Statistik Deskriptif
print("Statistik Deskriptif:")
print(data.describe())

# 2. Mengecek Missing Values
print("\nMissing Values:")
print(data.isnull().sum())

# 3. Visualisasi Distribusi Beberapa Variabel Kunci

# Distribusi Goal Ratio
plt.figure(figsize=(8, 6))
sns.histplot(data['Goal Ratio'], kde=True)
plt.title('Distribusi Goal Ratio')
plt.xlabel('Goal Ratio')
plt.ylabel('Frequency')
plt.show()

# Distribusi Total Value (dalam miliar atau jutaan)
plt.figure(figsize=(8, 6))
sns.histplot(data['Total Value'], kde=True)
plt.title('Distribusi Total Value')
plt.xlabel('Total Value')
plt.ylabel('Frequency')
plt.show()

# Distribusi Average Possession Club
plt.figure(figsize=(8, 6))
sns.histplot(data['Average Possession Club'], kde=True)
plt.title('Distribusi Average Possession Club')
plt.xlabel('Average Possession (%)')
plt.ylabel('Frequency')
plt.show()

# 4. Korelasi antar Variabel (hanya kolom numerik)
plt.figure(figsize=(12, 8))
numerical_data = data.select_dtypes(include='number')  # Pilih hanya kolom numerik
correlation_matrix = numerical_data.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Korelasi Antar Variabel')
plt.show()

# 5. Scatter Plot untuk Melihat Hubungan antara Variabel
# Contoh: Hubungan antara Average Possession dan Goal Ratio
plt.figure(figsize=(8, 6))
sns.scatterplot(x='Average Possession Club', y='Goal Ratio', data=data)
plt.title('Hubungan antara Average Possession dan Goal Ratio')
plt.xlabel('Average Possession (%)')
plt.ylabel('Goal Ratio')
plt.show()


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Load the dataset
file_path = 'cleaned_final_stats_with_team_and_venue_id.csv'
df = pd.read_csv(file_path)

# Step 1: Define the target variable based on Total Goal
# Create a binary target variable where 1 = Home Win, 0 = Away Win/Draw
df['Home Win'] = (df['Match Type'] == 'Home') & (df['Total Goal'] > df['GA'])
df['Home Win'] = df['Home Win'].astype(int)  # Convert to integer (1 for win, 0 for not)

# Step 2: Select features
# Using a set of features likely to influence match outcome
features = [
    'Shoot On Target', 'Shoot Off Target', 'Goal Ratio', 'Points', 'Rank',
    'Average Possession Club', 'Average Opponent Possession', 'Capacity'
]

# Ensure that any missing values are handled
df = df.dropna(subset=features + ['Home Win'])

# Step 3: Split data into train and test sets
X = df[features]
y = df['Home Win']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Initialize and train the model
model = RandomForestClassifier(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

# Step 5: Predict and evaluate
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability of Home Win

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
report = classification_report(y_test, y_pred)

# Display the results
print("Accuracy:", accuracy)
print("ROC-AUC:", roc_auc)
print("Classification Report:\n", report)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Load the dataset
file_path = 'cleaned_final_stats_with_team_and_venue_id.csv'
df = pd.read_csv(file_path)

# Step 1: Define the target variable with two categories: Home Win and Away Win
# - Home Win: 1 (if Total Goal home > Total Goal away)
# - Away Win: 0 (if Total Goal home < Total Goal away or it’s an away game)
df['Match Outcome'] = df.apply(
    lambda row: 1 if (row['Match Type'] == 'Home') and (row['Total Goal'] > row['GA']) else 0,
    axis=1
)

# Step 2: Select features
# Using a set of features likely to influence match outcome
features = [
    'Shoot On Target', 'Shoot Off Target', 'Goal Ratio', 'Points', 'Rank',
    'Average Possession Club', 'Average Opponent Possession', 'Capacity'
]

# Ensure that any missing values are handled
df = df.dropna(subset=features + ['Match Outcome'])

# Step 3: Split data into train and test sets
X = df[features]
y = df['Match Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Initialize and train the model
model = RandomForestClassifier(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

# Step 5: Predict probabilities for Home Win and Away Win in the test set
y_pred_proba = model.predict_proba(X_test)  # Probabilities for each class

# Create a DataFrame to store the fixture IDs, predicted probabilities, and actual outcomes
predictions = X_test.copy()
predictions['Fixture ID'] = df.loc[X_test.index, 'Fixture ID']
predictions['Home Win Probability'] = y_pred_proba[:, 1]  # Probability of Home Win (class 1)
predictions['Away Win Probability'] = y_pred_proba[:, 0]  # Probability of Away Win (class 0)
predictions['Actual Outcome'] = y_test.values  # Actual result

# Display results for each fixture in the test set, showing the Fixture ID and predicted win probabilities
output = predictions[['Fixture ID', 'Home Win Probability', 'Away Win Probability', 'Actual Outcome']].reset_index(drop=True)
print(output.head(10))  # Display top 10 rows for verification


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Step 1: Load the dataset
file_path = 'cleaned_final_stats_with_team_and_venue_id.csv'
data = pd.read_csv(file_path)

# Step 2: Separate data into Home and Away records
home_data = data[data['Match Type'] == 'Home']
away_data = data[data['Match Type'] == 'Away']

# Step 3: Merge Home and Away records for each Fixture ID
merged_data = pd.merge(home_data, away_data, on='Fixture ID', suffixes=('_home', '_away'))

# Define a function to get match result
def get_match_result(row):
    if row['GF_home'] > row['GA_away']:
        return 'Home Win'
    elif row['GF_home'] < row['GA_away']:
        return 'Away Win'
    else:
        return 'Draw'

# Apply function to create 'Match Result' column
merged_data['Match Result'] = merged_data.apply(get_match_result, axis=1)

# Step 4: Define features, including performance metrics and league position if available
features = [
    'Shoot Off Target_home', 'Shoot On Target_home', 'Goal Ratio_home',
    'Average Possession Club_home', 'Average Opponent Possession_home',
    'Points_home', 'Won_home', 'Drawn_home', 'Lost_home', 'GF_home', 'GA_home', 'GD_home',
    'Shoot Off Target_away', 'Shoot On Target_away', 'Goal Ratio_away',
    'Average Possession Club_away', 'Average Opponent Possession_away',
    'Points_away', 'Won_away', 'Drawn_away', 'Lost_away', 'GF_away', 'GA_away', 'GD_away'
    # Optionally add 'League Position_home' and 'League Position_away' if available
]

# Step 5: Prepare feature matrix X and target variable y
X = merged_data[features]
y = merged_data['Match Result']

# Step 6: Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Step 7: Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Step 8: Train the Random Forest model with updated features
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 9: Predict probabilities for each match outcome on all data
y_pred_proba = model.predict_proba(X)

# Step 10: Convert predictions to a DataFrame for better readability
proba_df = pd.DataFrame(y_pred_proba, columns=label_encoder.classes_)
proba_df['Fixture ID'] = merged_data['Fixture ID']
proba_df['Home Team'] = merged_data['Team_home']
proba_df['Away Team'] = merged_data['Team_away']
proba_df['Stadium'] = merged_data['Stadium_x_home']
proba_df['Points_home'] = merged_data['Points_home']
proba_df['Points_away'] = merged_data['Points_away']

# Step 11: Apply Selective Home Boost (5%) only if Home team has more points than Away team
home_boost_factor = 0.05
proba_df.loc[proba_df['Points_home'] > proba_df['Points_away'], 'Home Win'] += home_boost_factor

# Normalize probabilities to ensure they sum to 1
proba_df[['Away Win', 'Draw', 'Home Win']] = proba_df[
    ['Away Win', 'Draw', 'Home Win']
].div(proba_df[['Away Win', 'Draw', 'Home Win']].sum(axis=1), axis=0)

# Step 12: Convert probabilities to percentages and add '%' symbol
proba_df['Away Win'] = (proba_df['Away Win'] * 100).round(2).astype(str) + '%'
proba_df['Draw'] = (proba_df['Draw'] * 100).round(2).astype(str) + '%'
proba_df['Home Win'] = (proba_df['Home Win'] * 100).round(2).astype(str) + '%'

# Step 13: Save to CSV
output_file_path = 'fixture_predictions_with_performance_metrics.csv'
proba_df[['Fixture ID', 'Home Team', 'Away Team', 'Stadium', 'Away Win', 'Draw', 'Home Win']].to_csv(output_file_path, index=False)

print(f"Predictions saved to {output_file_path}")


Predictions saved to fixture_predictions_with_performance_metrics.csv


In [14]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
import numpy as np

# Langkah 1: Memuat dataset
file_path = 'scaled_final_stats_with_team_and_venue_id.csv'
data = pd.read_csv(file_path)

# Langkah 2: Pisahkan data menjadi catatan Home dan Away
home_data = data[data['Match Type'] == 'Home']
away_data = data[data['Match Type'] == 'Away']

# Langkah 3: Gabungkan catatan Home dan Away berdasarkan Fixture ID
merged_data = pd.merge(home_data, away_data, on='Fixture ID', suffixes=('_home', '_away'))

# Fungsi untuk mendapatkan hasil pertandingan
def get_match_result(row):
    if row['GF_home'] > row['GA_away']:
        return 'Home Win'
    elif row['GF_home'] < row['GA_away']:
        return 'Away Win'
    else:
        return 'Draw'

# Terapkan fungsi untuk membuat kolom 'Match Result'
merged_data['Match Result'] = merged_data.apply(get_match_result, axis=1)

# Langkah 4: Hitung rasio performa dan gunakan sebagai fitur
merged_data['Performance Ratio_home'] = (merged_data['Won_home'] + merged_data['Drawn_home']) / (
    merged_data['Won_home'] + merged_data['Drawn_home'] + merged_data['Lost_home']
)
merged_data['Performance Ratio_away'] = (merged_data['Won_away'] + merged_data['Drawn_away']) / (
    merged_data['Won_away'] + merged_data['Drawn_away'] + merged_data['Lost_away']
)

# Langkah 5: Tentukan fitur dan bobot untuk setiap fitur, termasuk bobot negatif untuk fitur tertentu
features = [
    'Shoot On Target_home', 'Shoot Off Target_home', 'Total Goal_home',
    'Average Possession Club_home','Average Opponent Possession_home', 'Total Value_home',
    'Points_home', 'Won_home', 'Lost_home', 'GF_home', 'GA_home', 
    'Performance Ratio_home', 'Average Opponent Possession_home',  # Rasio performa dan kepemilikan lawan
    'Shoot On Target_away', 'Shoot Off Target_away', 'Total Goal_away',
    'Average Possession Club_away','Average Opponent Possession_away', 'Total Value_away',
    'Points_away', 'Won_away', 'Lost_away', 'GF_away', 'GA_away',
    'Performance Ratio_away', 'Average Opponent Possession_away'
]

# Menentukan bobot, termasuk bobot negatif untuk fitur-fitur tertentu
feature_weights = {
    'Shoot On Target_home': 1.0, 'Shoot Off Target_home': 0.8, 'Total Goal_home': 1.2,
    'Average Possession Club_home': 0.7,'Total Value_home': 0.5,
    'Points_home': 1.5, 'Won_home': 1.2, 'Lost_home': -1.2,  # Lost memiliki bobot negatif
    'GF_home': 1.1, 'GA_home': -1.1, 
    'Performance Ratio_home': 1.4, 'Average Opponent Possession_home': -0.7,  # Bobot negatif
    'Shoot On Target_away': 1.0, 'Shoot Off Target_away': 0.8, 'Total Goal_away': 1.2,
    'Average Possession Club_away': 0.7, 'Total Value_away': 0.5,
    'Points_away': 1.5, 'Won_away': 1.2, 'Lost_away': -1.2,  # Lost dengan bobot negatif
    'GF_away': 1.1, 'GA_away': -1.1, 
    'Performance Ratio_away': 1.4, 'Average Opponent Possession_away': -0.7  # Bobot negatif
}

# Terapkan bobot pada setiap fitur, termasuk bobot negatif
for feature in features:
    merged_data[feature] = merged_data[feature] * feature_weights[feature]

# Langkah 6: Siapkan matriks fitur X dan variabel target y
X = merged_data[features]
y = merged_data['Match Result']

# Langkah 7: Normalisasi fitur
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode label target
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Langkah 8: Membagi data menggunakan StratifiedKFold untuk menjaga keseimbangan
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for train_index, test_index in skf.split(X, y_encoded):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y_encoded[train_index], y_encoded[test_index]

# Langkah 9: Latih model Random Forest dengan pembobotan kelas yang sesuai
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,
    random_state=42,
    class_weight='balanced'
)
model.fit(X_train, y_train)

# Evaluasi model pada data uji untuk memeriksa keseimbangan
y_test_pred = model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_))

# Langkah 10: Prediksi probabilitas untuk setiap pertandingan
y_pred_proba = model.predict_proba(X)

# Langkah 11: Konversi hasil ke DataFrame untuk kemudahan pembacaan
proba_df = pd.DataFrame(y_pred_proba, columns=label_encoder.classes_)
proba_df['Fixture ID'] = merged_data['Fixture ID']
proba_df['Home Team'] = merged_data['Team_home']
proba_df['Away Team'] = merged_data['Team_away']
proba_df['Home Team ID'] = merged_data['Team ID_home']
proba_df['Away Team ID'] = merged_data['Team ID_away']
proba_df['Points_home'] = merged_data['Points_home']
proba_df['Points_away'] = merged_data['Points_away']

# Langkah 12: Terapkan batas minimum untuk Draw dan Home Boost jika diperlukan
draw_min_limit = 0.30
home_boost_factor = 0.01

for i, row in proba_df.iterrows():
    home_win_prob = row['Home Win']
    draw_prob = row['Draw']
    away_win_prob = row['Away Win']
    
    if draw_prob < draw_min_limit:
        draw_prob = draw_min_limit
        
    remaining_prob = 1 - draw_prob
    home_win_prob = remaining_prob * (home_win_prob / (home_win_prob + away_win_prob))
    away_win_prob = remaining_prob * (away_win_prob / (home_win_prob + away_win_prob))
    
    if row['Points_home'] > row['Points_away']:
        home_win_prob += home_boost_factor
    
    total_prob = home_win_prob + draw_prob + away_win_prob
    home_win_prob /= total_prob
    draw_prob /= total_prob
    away_win_prob /= total_prob
    
    proba_df.at[i, 'Home Win'] = home_win_prob
    proba_df.at[i, 'Draw'] = draw_prob
    proba_df.at[i, 'Away Win'] = away_win_prob

# Langkah 13: Konversi probabilitas ke dalam bentuk persentase
proba_df['Home Win'] = (proba_df['Home Win'] * 100).round(2).astype(str) + '%'
proba_df['Draw'] = (proba_df['Draw'] * 100).round(2).astype(str) + '%'
proba_df['Away Win'] = (proba_df['Away Win'] * 100).round(2).astype(str) + '%'

# Langkah 14: Simpan hasil prediksi ke dalam CSV
output_file_path = 'fixture_predictions.csv'
proba_df[['Fixture ID', 'Home Team', 'Away Team', 'Home Team ID', 'Away Team ID', 'Home Win', 'Draw', 'Away Win']].to_csv(output_file_path, index=False)

print(f"Prediksi disimpan dalam {output_file_path}")


c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\skl

Classification Report:
              precision    recall  f1-score   support

    Away Win       0.93      0.93      0.93        29
        Draw       0.00      0.00      0.00         1
    Home Win       0.96      0.98      0.97        46

    accuracy                           0.95        76
   macro avg       0.63      0.64      0.63        76
weighted avg       0.93      0.95      0.94        76

Prediksi disimpan dalam fixture_predictions.csv


In [4]:
# Langkah 1: Memuat dataset
file_path = 'scaled_final_stats_with_team_and_venue_id.csv'
data = pd.read_csv(file_path)

# Tampilkan semua kolom dalam dataset untuk memverifikasi nama kolom
print("Columns in dataset:", data.columns)

# Langkah 2 dan seterusnya tetap sama...


Columns in dataset: Index(['Club', 'Stadium_x', 'Capacity', 'Average Attendance',
       'Shoot Off Target', 'Shoot On Target', 'Total Goal', 'Goal Ratio',
       'Rank', 'Total Value', 'Played', 'Won', 'Drawn', 'Lost', 'GF', 'GA',
       'GD', 'Points', 'Games Played', 'Average Possession Club',
       'Average Opponent Possession', 'Team ID', 'Team', 'Venue ID',
       'Fixture ID', 'Match Type', 'Game Date', 'Stadium_y'],
      dtype='object')


### Data All Player sumber github


In [5]:
import pandas as pd

# Membaca file CSV (dengan menangani baris buruk)
file_path = 'merged_gw.csv'
data = pd.read_csv(file_path, on_bad_lines='skip')

# 1. Hapus duplikat jika ada
data_cleaned = data.drop_duplicates()

# 2. Hapus baris dengan terlalu banyak nilai kosong (> 50% kolom kosong)
data_cleaned = data_cleaned.dropna(thresh=len(data_cleaned.columns) * 0.5)

# 3. Isi nilai kosong yang tersisa dengan nilai default
data_cleaned = data_cleaned.fillna(value={col: "default_value" for col in data_cleaned.columns})

# 4. Ubah format kolom 'kickoff_time' ke format datetime jika ada
if 'kickoff_time' in data_cleaned.columns:
    data_cleaned['kickoff_time'] = pd.to_datetime(data_cleaned['kickoff_time'], errors='coerce')

# 5. Mengganti nama tim yang tidak sesuai (menangani masalah seperti spasi ekstra dan tanda kutip)
team_name_mapping = {
    "Nott'm Forest": "Nottingham Forest",
}

# Gunakan strip() dan replace() untuk membersihkan nama tim dari karakter yang tidak terlihat
data_cleaned['team'] = data_cleaned['team'].str.strip().replace(team_name_mapping)

# 6. Menggabungkan nilai pemain dengan nama sama (agregasi numerik) dan mengurutkan berdasarkan klub
if 'name' in data_cleaned.columns and 'team' in data_cleaned.columns:
    numeric_columns = ['minutes','goals_scored', 'assists', 'clean_sheets', 'yellow_cards', 'red_cards']
    selected_columns = ['name', 'team', 'position'] + numeric_columns
    data_selected = data_cleaned[selected_columns]
    data_combined = data_selected.groupby(['name', 'team', 'position'], as_index=False).sum()

    # Urutkan berdasarkan klub (team)
    data_combined = data_combined.sort_values(by='team').reset_index(drop=True)

    # Menentukan urutan posisi pemain
    position_order = ['GK', 'DEF', 'MID', 'FWD']
    data_combined['position'] = pd.Categorical(data_combined['position'], categories=position_order, ordered=True)

    # Mengurutkan berdasarkan posisi setelah klub
    data_combined = data_combined.sort_values(by=['team', 'position']).reset_index(drop=True)

# 7. Simpan data yang sudah dibersihkan ke file baru
output_path = 'cleaned_merged_gw.csv'
data_combined.to_csv(output_path, index=False)

print("Data berhasil dibersihkan, digabungkan, dan diurutkan berdasarkan klub dan posisi.")
print(f"Data disimpan ke: {output_path}")

# Menampilkan sebagian data untuk verifikasi
print(data_combined.head())


Data berhasil dibersihkan, digabungkan, dan diurutkan berdasarkan klub dan posisi.
Data disimpan ke: cleaned_merged_gw.csv
                   name     team position  minutes  goals_scored  assists  \
0         Tommy Setford  Arsenal       GK        0             0        0   
1  Norberto Murara Neto  Arsenal       GK        0             0        0   
2     David Raya Martin  Arsenal       GK     1260             0        0   
3        Aaron Ramsdale  Arsenal       GK        0             0        0   
4             Karl Hein  Arsenal       GK        0             0        0   

   clean_sheets  yellow_cards  red_cards  
0             0             0          0  
1             0             0          0  
2             5             2          0  
3             0             0          0  
4             0             0          0  


In [2]:
print(data_cleaned['team'].unique())


['Bournemouth' 'Nottingham Forest' 'Arsenal' 'Crystal Palace'
 'Southampton' 'Aston Villa' 'Spurs' 'Wolves' 'Brentford' 'Chelsea'
 'Brighton' 'Leicester' 'Ipswich' 'Man Utd' 'West Ham' 'Fulham'
 'Liverpool' 'Newcastle' 'Man City' 'Everton']


### mengolah statstik performa tim 
